In [1]:
# 아이투자
# 에프엔가이드
# 세종기업데이터
# 네이버
# DART


In [1]:
# Read urls from Excel file
# Parse information from itooza
# Write information to Excel file
import pandas as pd
import numpy as np
import os, sys, time
import sqlite3

import selenium
import xlrd
import xlsxwriter
from bs4 import BeautifulSoup
import urllib.request
import pickle
import getopt

In [2]:
from selenium import webdriver

In [3]:
driver = webdriver.Chrome('/Users/DOWAN/Downloads/chromedriver_win32/chromedriver')
#driver = webdriver.PhantomJS('C:\Users\DOWAN\Downloads\phantomjs-2.1.1-windows\bin\phantomjs')
driver.implicitly_wait(3)

In [4]:
driver.get('https://login.itooza.com/login.htm?qurl64=aHR0cDovL3d3dy5pdG9vemEuY29tLw==')
# 수작업으로 로그인한다.

In [ ]:
# # mode= 연간: dy, 분기: db
# # 재무상태표
# http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2  # 개별
# http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1  # 연결
# # 손익계산서
# http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2  # 개별
# http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1  # 연결
# # 현금흐름표
# http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=4&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2 # 개별
# http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=4&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1 # 연결

In [14]:
# 종목데이터 DB불러오기 
# kospi_df= None
kosdaq_df= None
with sqlite3.connect('./data/StockData.sqlite') as conn:
    #kospi_df= pd.read_sql("SELECT * From '코스피 종목코드'", con=conn) # if_exists='append'
    kosdaq_df= pd.read_sql("SELECT * From '코스닥 종목코드'", con=conn) # if_exists='append'

In [15]:
print(kosdaq_df.shape)
kosdaq_df.head()

(1309, 9)


,종목코드,종목명,구분,소속,업종,업종명,KOSPI200,상태,상장일
0,A000250,삼천당제약,1,2,266,(코스닥)제약,0,0,20001004
1,A000440,중앙에너비스,1,2,227,(코스닥)유통,0,0,19930324
2,A001000,신라섬유,1,2,212,(코스닥)기타,0,0,19940628
3,A001540,안국약품,1,2,266,(코스닥)제약,0,0,20000615
4,A001810,무림SP,1,2,262,"(코스닥)종이, 목재",0,0,20000104


In [16]:
kosdaq_df[kosdaq_df['종목코드']== 'A237690']
kosdaq_df= kosdaq_df[1186:]
kosdaq_df.tail(10)

,종목코드,종목명,구분,소속,업종,업종명,KOSPI200,상태,상장일
1299,A307160,하나머스트제6호스팩,1,2,231,(코스닥)금융,0,0,20181130
1300,A307180,신영스팩4호,1,2,231,(코스닥)금융,0,0,20181221
1301,A307280,교보8호스팩,1,2,231,(코스닥)금융,0,0,20181205
1302,A307750,대신밸런스제6호스팩,1,2,231,(코스닥)금융,0,0,20181219
1303,A307870,골든브릿지이안5호,1,2,231,(코스닥)금융,0,0,20181212
1304,A309930,삼성머스트스팩3호,1,2,231,(코스닥)금융,0,0,20181220
1305,A310200,미래에셋대우스팩2호,1,2,231,(코스닥)금융,0,0,20181221
1306,A310840,엔에이치스팩13호,1,2,231,(코스닥)금융,0,0,20181224
1307,A310870,한국제8호스팩,1,2,231,(코스닥)금융,0,0,20181220
1308,A311270,키움제5호스팩,1,2,231,(코스닥)금융,0,0,20181226


In [12]:
code_cnt= 0

for code in kosdaq_df['종목코드']:
    name= kosdaq_df[kosdaq_df['종목코드']== code]['종목명'].values[0]
    code= code[1:]    
    url= []
    target_table= []
    code_cnt= code_cnt+1

#     #  연간 : 재무상태표(개별,연결), 손익계산서(개별,연결)
#     url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2" % (code))
#     url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code))
#     url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2" % (code))
#     url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code))
#     #  분기 : 재무상태표(개별,연결), 손익계산서(개별,연결)
#     url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2" % (code))
#     url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code))
#     url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2" % (code))
#     url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code))

    #  연간 : 재무상태표(연결), 손익계산서(연결), 재무상태표(개별), 손익계산서(개별)
    url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code))
    url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code))
    url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2" % (code))
    url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2" % (code))

    #  분기 : 재무상태표(연결), 손익계산서(연결), 재무상태표(개별), 손익계산서(개별)
    url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code))
    url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code))
    url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2" % (code))
    url.append("http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=2&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=2" % (code))


    for i in range(8):
        status= True
        try:
            driver.get(url[i])
            driver.implicitly_wait(1)
            time.sleep(0.5)
            html= driver.page_source
        except Exception as e: # 조회종목 웹페이지 없을때 처리
            print(e)
            status= False
            break
        if status == True:
            print('>', end='')
            soup= BeautifulSoup(html, "html.parser")  
            target= soup.find("div", id="a01")
            target_table.append(str(target))

            
   
    # 페이지 저장
    if (status == True) or (i == 7):
        print(' ',code,'조회완료')
        with open('./data_doc/doc_%s.html' %(code), 'w', encoding='utf-8') as f:
            f.write('\n<meta charset="utf-8">\n')
            f.write('\n\n%s 재무상태표 연결 연간\n\n' %(name))
            f.write(target_table[0])
            f.write('\n\n%s 손익계산서 연결 연간\n\n' %(name))
            f.write(target_table[1])
            f.write('\n\n%s 재무상태표 개별 연간\n\n' %(name))
            f.write(target_table[2])
            f.write('\n\n%s 손익계산서 개별 연간\n\n' %(name))
            f.write(target_table[3])
            f.write('\n\n%s 재무상태표 연결 분기\n\n' %(name))
            f.write(target_table[4])
            f.write('\n\n%s 손익계산서 연결 분기\n\n' %(name))
            f.write(target_table[5])
            f.write('\n\n%s 재무상태표 개별 분기\n\n' %(name))
            f.write(target_table[6])
            f.write('\n\n%s 손익계산서 개별 분기\n\n' %(name))
            f.write(target_table[7])
            ##################################################
            url_b= 'http://search.itooza.com/index.htm?seName=%s' % (code)
            handle_b= urllib.request.urlopen(url_b).read()
            html_b = handle_b.decode('euc-kr','replace').encode('utf-8','replace')
            soup_b = BeautifulSoup(html_b, 'html.parser')
            table= soup_b.find_all("div", id="indexTable2")[0].find('table')
            f.write('\n\n%s 투자지표 연간\n\n' %(name))
            f.write('\n<div id="indexTable">\n')
            f.write(str(table))
            f.write('\n</div>\n')
            table= soup_b.find_all("div", id="indexTable3")[0].find('table')
            f.write('\n\n%s 투자지표 분기\n\n' %(name))
            f.write('\n<div id="indexTable">\n')
            f.write(str(table))
            f.write('\n</div>\n')
            ##################################################            
    else:
        print(' ',code,'조회실패')
        alert= driver.switch_to_alert()
        #alert.accept() #for two buttons, choose the affirmative one
        alert.dismiss() #to cancel the affirmative decision

print('종목조회개수:',code_cnt)
# 72.0.3626.109
    

>>>>>>>>  237750 조회완료
>>>>>>>>  237880 조회완료
>>>>>>>>  238090 조회완료
>>>>>>>>  238120 조회완료
>>>>>>>>  238200 조회완료
>>>>>>>>  238490 조회완료
>>>>>>>>  239340 조회완료
>>>>>>>>  239610 조회완료
>>>>>>>>  240810 조회완료
>>>>>>>>  241520 조회완료
>>>>>>>>  241690 조회완료
>>>>>>>>  241710 조회완료
>>>>>>>>  241770 조회완료
>>>>>>>>  241790 조회완료
>>>>>>>>  241820 조회완료
>>>>>>>>  242040 조회완료
>>>>>>>>  243070 조회완료
>>>>>>>>  243840 조회완료
>>>>>>>>  245620 조회완료
>>>>>>>>  246690 조회완료
>>>>>>>>  246710 조회완료
>>>>>>>>  246720 조회완료
>>>>>>>>  246960 조회완료
>>>>>>>>  250000 조회완료
>>>>>>>>  250060 조회완료
>>>>>>>>  250930 조회완료
>>>>>>>>  251370 조회완료
>>>>>>>>  251630 조회완료
>>>>>>>>  252500 조회완료
>>>>>>>>  253450 조회완료
>>>>>>>>  253590 조회완료
>>>>>>>>  254120 조회완료
>>>>>>>>  255220 조회완료
>>>>>>>>  255440 조회완료
>>>>>>>>  256630 조회완료
>>>>>>>>  256840 조회완료
>>>>>>>>  256940 조회완료
>>>>>>>>  257370 조회완료
>>>>>>>>  257730 조회완료
>>>>>>>>  258610 조회완료
>>>>>>>>  258790 조회완료
>>>>>>>>  258830 조회완료
>>>>>>>>  259630 조회완료
>>>>>>>>  260660 조회완료
>>>>>>>>  260930 조회완료
>>>>>>>>  

In [49]:
driver.quit()

In [39]:
#soup = BeautifulSoup(html, 'html.parser')

In [17]:
# code= '005490'
# #url='http://asp01.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&NewMenuID=11&cID=50&MenuYn=N'% (code)  #FN가이드
# #url = "http://search.itooza.com/index.htm?seName=%s" % (code)
  
# # 재무상태표
# url = "http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=dy&ss=10&sv=1&lsmode=1&accmode=1&pmode=2&exmode=1&lkmode=1" % (code)

# handle = urllib.request.urlopen(url).read()
# content = handle.decode('euc-kr','replace').encode('utf-8','replace')
# soup = BeautifulSoup(content, "html.parser")
# soup = BeautifulSoup(data, 'html.parser', from_encoding='utf-8') # 글자깨짐
# soup

In [55]:
# 재무파일 로딩
with open('./data_doc/doc_000020.html', 'r', encoding='utf-8') as f:
    soup= BeautifulSoup(f.read(), "html.parser")

target_table= []
for i in soup.find_all("div", id="a01"):
    target_table.append(i)
for i in soup.find_all("div", id="indexTable"):
    target_table.append(i)


In [61]:
# target_table[0] # 재무상태표 연결 연간
# target_table[1] # 손익계산서 연결 연간
# target_table[2] # 재무상태표 개별 연간
# target_table[3] # 손익계산서 개별 연간
# target_table[4] # 재무상태표 연결 분기
# target_table[5] # 손익계산서 연결 분기
# target_table[6] # 재무상태표 개별 분기
# target_table[7] # 손익계산서 개별 분기
# target_table[8] # 투자지표 연간
# target_table[9] # 투자지표 분기
target_table[7]

<div id="a01">
<table>
<tbody><tr><!-- 재무상태표, 손익계산서, 현금흐름표 공통 -->
<td style="width:180px;vertical-align:top;">
<style type="text/css">
								/* IE 7 및 하위버전 : 왼쪽메뉴에 스크롤 생기는 것.. width 오류가 있어서 호환성 보기 했을 경우에만 숨김.  */
								*:first-child+html a01 { overflow-x:hidden }
								</style>
<div class="a01" style="">
<table class="data-gray over1 treeTable" id="y10_tb_1">
<thead>
<tr height="22">
<th class="small t" scope="col">손익계산서</th><td style="display:none"></td>
</tr>
</thead>
<tbody id="tree_node">
<tr class="initialized parent collapsed" height="22" id="node-1">
<th nowrap="" scope="row" style="font-weight:bold"><span class="expander" style="margin-left: -12px; padding-left: 12px"></span>매출액(수익)</th>
<td style="display:none"></td>
</tr>
<tr class="depth2 child-of-node-1 collapsed" height="22" id="node-1-1" style="display: none;">
<th nowrap="" scope="row" style="padding: 0px 10px 0px 14px;">내수</th>
<td style="display:none"></td>
</tr>
<tr class="depth2 child-of-node-1 collapsed" heigh

In [ ]:
# ##########   개별 연간
# 자산총계 node-79
# 부채총계 node-125
# 자본총계 node-153
# 비지배주주지분 node-152
# 지배주주지분 node-127
# 자본금 node-128

# ##########   연결 연간
# 자산총계 node-80 ***
# 부채총계 node-126 ***
# 자본총계 node-153
# 비지배주주지분 node-152
# 지배주주지분 node-127
# 자본금 node-128

# ##########   개별 연간
# 매출액 node-1
# 영업이익 node-40 
# 당기순이익 node-126
# 지배주주순이익 , 없음
# 비지배주주순이익, 없음
# 주가 node-168
# 시가총액 node-169

# ##########   연결 연간
# 매출액 node-1
# 영업이익 node-39 ***
# 당기순이익 node-125 ***
# 지배지분 순이익 node-126 ***
# 비지배지분 순이익 node-127 ***
# 주가 node-166 ***
# 시가총액 node-167 ***


In [93]:
for i in [0,1,2,3]:
    cnt= 0
    date_연간=[]
    for thead in target_table[i].find_all('thead'):
        for span in thead.find_all('span'):
            value= span.text.strip().replace('.','')
            # print(value)
            date_연간.append(value)
            cnt= cnt+1
    print('date 총개수:',cnt)
    

for i in [4,5,6,7]:
    cnt= 0
    date_분기=[]
    for thead in target_table[i].find_all('thead'):
        for span in thead.find_all('span'):
            value= span.text.strip().replace('.','')
            # print(value)
            date_분기.append(value)
            cnt= cnt+1
    print('date 총개수:',cnt)
    
    

date 총개수: 21
date 총개수: 21
date 총개수: 21
date 총개수: 21
date 총개수: 84
date 총개수: 84
date 총개수: 84
date 총개수: 84


In [75]:
# ##########   개별 연간
value_list=[]
cnt= 0
col_title=['자산총계','부채총계', '자본총계','지배주주지분','비지배주주지분','자본금']
col_list=['node-79','node-125','node-153','node-127','node-152','node-128']
df_재무개별연간= pd.DataFrame()
df_재무개별연간['연간']= pd.Series(date_연간)

for col, tit in zip(col_list, col_title):
    value_list=[]
    cnt= 0
    title= target_table[0].find_all('tr', id= col)[0]
    title= title.find('th').text.strip()
    #print(title)
    if title != tit:
        print('계정항목 불일치', title)
        continue
    tr= target_table[0].find_all('tr', id= col)[1]
    for td in tr.find_all('td'):     
        value= td.text.strip().replace(',','')
        if value!='':
            #print(value)
            value_list.append(value)
            cnt= cnt+1
    df_재무개별연간[tit]= pd.Series(value_list)
    print(tit, '총개수:', cnt)

    
# ##########   연결 연간
value_list=[]
cnt= 0
col_title=['자산총계','부채총계', '자본총계','지배주주지분','비지배주주지분','자본금']
col_list=['node-80','node-126','node-153','node-127','node-152','node-128']
df_재무연결연간= pd.DataFrame()
df_재무연결연간['연간']= pd.Series(date_연간)

for col, tit in zip(col_list, col_title):
    value_list=[]
    cnt= 0
    title= target_table[1].find_all('tr', id= col)[0]
    title= title.find('th').text.strip()
    #print(title)
    if title != tit:
        print('계정항목 불일치', title)
        continue
    tr= target_table[1].find_all('tr', id= col)[1]
    for td in tr.find_all('td'):     
        value= td.text.strip().replace(',','')
        if value!='':
            #print(value)
            value_list.append(value)
            cnt= cnt+1
    df_재무연결연간[tit]= pd.Series(value_list)
    print(tit, '총개수:', cnt)    
        

자산총계 총개수: 21
부채총계 총개수: 21
자본총계 총개수: 21
비지배주주지분 총개수: 21
지배주주지분 총개수: 21
자본금 총개수: 21


In [90]:
# ##########   개별 연간
value_list=[]
cnt= 0
col_title=['매출액(수익)','영업이익','당기순이익','주가','시가총액']
col_list=['node-1','node-40','node-126','node-168','node-169']
df_손익개별연간= pd.DataFrame()
df_손익개별연간['연간']= pd.Series(date_연간)

for col, tit in zip(col_list, col_title):
    value_list=[]
    cnt= 0
    title= target_table[2].find_all('tr', id= col)[0]
    title= title.find('th').text.strip()
    #print(title)
    if title != tit:
        print('계정항목 불일치', title)
        continue
    tr= target_table[2].find_all('tr', id= col)[1]
    for td in tr.find_all('td'):     
        value= td.text.strip().replace(',','')
        if value!='':
            #print(value)
            value_list.append(value)
            cnt= cnt+1
    df_손익개별연간[tit]= pd.Series(value_list)
    print(tit, '총개수:', cnt)

    
# ##########   연결 연간
value_list=[]
cnt= 0
col_title=['매출액(수익)','영업이익', '당기순이익','지배지분 순이익','비지배지분 순이익','주가','시가총액']
col_list=['node-1','node-39','node-125','node-126','node-127','node-166','node-167']
df_손익연결연간= pd.DataFrame()
df_손익연결연간['연간']= pd.Series(date_연간)

for col, tit in zip(col_list, col_title):
    value_list=[]
    cnt= 0
    title= target_table[3].find_all('tr', id= col)[0]
    title= title.find('th').text.strip()
    #print(title)
    if title != tit:
        print('계정항목 불일치', title)
        continue
    tr= target_table[3].find_all('tr', id= col)[1]
    for td in tr.find_all('td'):     
        value= td.text.strip().replace(',','')
        if value!='':
            #print(value)
            value_list.append(value)
            cnt= cnt+1
    df_손익연결연간[tit]= pd.Series(value_list)
    print(tit, '총개수:', cnt) 

    

매출액(수익) 총개수: 21
영업이익 총개수: 21
당기순이익 총개수: 21
주가 총개수: 21
시가총액 총개수: 21
매출액(수익) 총개수: 21
영업이익 총개수: 21
당기순이익 총개수: 21
지배지분 순이익 총개수: 21
비지배지분 순이익 총개수: 21
주가 총개수: 21
시가총액 총개수: 21


In [94]:
# ##########   개별 분기
value_list=[]
cnt= 0
col_title=['자산총계','부채총계', '자본총계','지배주주지분','비지배주주지분','자본금']
col_list=['node-79','node-125','node-153','node-127','node-152','node-128']
df_재무개별분기= pd.DataFrame()
df_재무개별분기['분기']= pd.Series(date_분기)

for col, tit in zip(col_list, col_title):
    value_list=[]
    cnt= 0
    title= target_table[4].find_all('tr', id= col)[0]
    title= title.find('th').text.strip()
    #print(title)
    if title != tit:
        print('계정항목 불일치', title)
        continue
    tr= target_table[4].find_all('tr', id= col)[1]
    for td in tr.find_all('td'):     
        value= td.text.strip().replace(',','')
        if value!='':
            #print(value)
            value_list.append(value)
            cnt= cnt+1
    df_재무개별분기[tit]= pd.Series(value_list)
    print(tit, '총개수:', cnt)

    
# ##########   연결 연간
value_list=[]
cnt= 0
col_title=['자산총계','부채총계', '자본총계','지배주주지분','비지배주주지분','자본금']
col_list=['node-80','node-126','node-153','node-127','node-152','node-128']
df_재무연결분기= pd.DataFrame()
df_재무연결분기['연간']= pd.Series(date_분기)

for col, tit in zip(col_list, col_title):
    value_list=[]
    cnt= 0
    title= target_table[5].find_all('tr', id= col)[0]
    title= title.find('th').text.strip()
    #print(title)
    if title != tit:
        print('계정항목 불일치', title)
        continue
    tr= target_table[5].find_all('tr', id= col)[1]
    for td in tr.find_all('td'):     
        value= td.text.strip().replace(',','')
        if value!='':
            #print(value)
            value_list.append(value)
            cnt= cnt+1
    df_재무연결분기[tit]= pd.Series(value_list)
    print(tit, '총개수:', cnt)    
        

자산총계 총개수: 84
부채총계 총개수: 84
자본총계 총개수: 84
지배주주지분 총개수: 84
비지배주주지분 총개수: 84
자본금 총개수: 84
자산총계 총개수: 84
부채총계 총개수: 84
자본총계 총개수: 84
지배주주지분 총개수: 84
비지배주주지분 총개수: 84
자본금 총개수: 84


In [97]:
# ##########   개별 연간
value_list=[]
cnt= 0
col_title=['매출액(수익)','영업이익','당기순이익','주가','시가총액']
col_list=['node-1','node-40','node-126','node-168','node-169']
df_손익개별분기= pd.DataFrame()
df_손익개별분기['분기']= pd.Series(date_분기)

for col, tit in zip(col_list, col_title):
    value_list=[]
    cnt= 0
    title= target_table[6].find_all('tr', id= col)[0]
    title= title.find('th').text.strip()
    #print(title)
    if title != tit:
        print('계정항목 불일치', title)
        continue
    tr= target_table[6].find_all('tr', id= col)[1]
    for td in tr.find_all('td'):     
        value= td.text.strip().replace(',','')
        if value!='':
            #print(value)
            value_list.append(value)
            cnt= cnt+1
    df_손익개별분기[tit]= pd.Series(value_list)
    print(tit, '총개수:', cnt)

    
# ##########   연결 연간
value_list=[]
cnt= 0
col_title=['매출액(수익)','영업이익', '당기순이익','지배지분 순이익','비지배지분 순이익','주가','시가총액']
col_list=['node-1','node-39','node-125','node-126','node-127','node-166','node-167']
df_손익연결분기= pd.DataFrame()
df_손익연결분기['분기']= pd.Series(date_분기)

for col, tit in zip(col_list, col_title):
    value_list=[]
    cnt= 0
    title= target_table[7].find_all('tr', id= col)[0]
    title= title.find('th').text.strip()
    #print(title)
    if title != tit:
        print('계정항목 불일치', title)
        continue
    tr= target_table[7].find_all('tr', id= col)[1]
    for td in tr.find_all('td'):     
        value= td.text.strip().replace(',','')
        if value!='':
            #print(value)
            value_list.append(value)
            cnt= cnt+1
    df_손익연결분기[tit]= pd.Series(value_list)
    print(tit, '총개수:', cnt) 


매출액(수익) 총개수: 84
영업이익 총개수: 84
당기순이익 총개수: 84
주가 총개수: 84
시가총액 총개수: 84
매출액(수익) 총개수: 84
영업이익 총개수: 84
당기순이익 총개수: 84
지배지분 순이익 총개수: 84
비지배지분 순이익 총개수: 84
주가 총개수: 84
시가총액 총개수: 84


In [182]:
# 235 * 12 * 2000 = 5640000
# 235 * 8 * 2000 = 3760000
# 코스피 종목
# 재무상태표(연결,개별,분기,연간), 손익계산서(연결,개별,분기,연간)
# 1000 * 8 * 0.5 / 60 / 60 =1.1

import locale
locale.setlocale( locale.LC_ALL, 'english_USA' )
locale.atof('123')


123.0

In [74]:
# with open('./daw1.txt', 'r', encoding='utf-8') as f:
#     soup= BeautifulSoup(f.read(), "html.parser")
# #soup

﻿
<!DOCTYPE html>

<html lang="ko">
<head>
<link href="../css/compeach.css?_=20190115" rel="stylesheet"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<!-- <meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0" name="viewport"> -->
<script src="../js/lib/jquery-1.11.1.min.js" type="text/javascript"></script>
<script src="../js/comp_load.js?ver=1.1" type="text/javascript"></script>
<script src="../js/lib/jquery.autocomplete.js" type="text/javascript"></script>
<link href="../css/jquery.autocomplete.css" rel="stylesheet" type="text/css">
<link href="/SVO2/img/CompanyGuide.ico" rel="SHORTCUT ICON"/>
<script src="../js/autocomplete.js" type="text/javascript"></script>
<meta content="" name="author"/>
<meta content="" name="description"/>
<title>다우기술(A023590) | Snapshot | 기업정보 | Company Guide</title>
<script>
///SVO2/xml/Snapshot_all/023590.xml
//제조
$( window ).resize(fun